In [1]:
# --- Make repo imports (app/, agents/, data/, shared/) work inside this notebook ---
import os, sys, pathlib, subprocess

def find_repo_root(start: pathlib.Path | None = None) -> pathlib.Path | None:
    """
    Find the repo root by (in order):
    1) GIT root if available
    2) Presence of requirements/pyproject + at least one of the new top-level dirs
    """
    start = start or pathlib.Path().resolve()

    # 1) Try git
    try:
        git_root = subprocess.check_output(
            ["git", "rev-parse", "--show-toplevel"],
            cwd=str(start),
            text=True
        ).strip()
        p = pathlib.Path(git_root)
        if (p / "requirements.txt").exists() or (p / "pyproject.toml").exists():
            return p
    except Exception:
        pass

    # 2) Walk upwards looking for new layout markers
    markers = {"app", "agents", "data", "shared"}
    for p in [start, *start.parents]:
        has_manifest = (p / "requirements.txt").exists() or (p / "pyproject.toml").exists()
        has_any_marker = any((p / m).exists() for m in markers)
        if has_manifest and has_any_marker:
            return p
    return None

root = find_repo_root()
if root is None:
    raise RuntimeError(
        "Could not locate repo root. Ensure this notebook is inside the repo "
        "or set REPO_ROOT to your repo path."
    )

# Optional: allow manual override via env var
root = pathlib.Path(os.getenv("REPO_ROOT", str(root))).resolve()

# 1) Put repo root at the front of sys.path so `import agents`, `import shared`, etc. work
root_str = str(root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

# 2) Switch working dir to repo root so relative paths work
os.chdir(root_str)

print("Repo root:", root_str)
print("Python exe:", sys.executable)
print("sys.path[0]:", sys.path[0])


Repo root: C:\Users\reube\OneDrive\Desktop\Kaggle\geomarket-insight
Python exe: C:\Users\reube\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
sys.path[0]: C:\Users\reube\OneDrive\Desktop\Kaggle\geomarket-insight


In [3]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\reube\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:

from agents.workflows.query_workflow import run_slot_agent

q = "show me neighbourhoods with no cafes within 500 meters and income > 70000"
out = run_slot_agent(q)
out

{'intent': 'gap',
 'target_category': 'coffee_shop',
 'metrics': [],
 'dimensions': [{'name': 'tract'}],
 'filters': [{'field': 'distance', 'op': 'within_km', 'value': '0.5'},
  {'field': 'income', 'op': 'gt', 'value': '70000'}]}